<a href="https://colab.research.google.com/github/Noufeine/conway_99_graph/blob/main/3-g%C3%A9n%C3%A9tique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import datetime
import os, psutil


#PARAMETRE
SIZE = 16
DEGRE = 6
LAMBDA = 2
MU = 2


#fonction qui renvoie tous les voisins sauf le exepct
def tous_les_voisins(matrice, sommet, sauf):
  tous_voisins = []
  for i in range(len(matrice[sommet])):
    if(matrice[sommet][i] == 1 and i != sauf):
      tous_voisins.append(i)
  return tous_voisins

#renvoie le nombre de voisins en commun entre 2 sommets:
def nbr_voisins_en_commun(matrice, sommet_1, sommet_2):
  les_voisins_1 = tous_les_voisins(matrice, sommet_1, sommet_2)
  les_voisins_2 = tous_les_voisins(matrice, sommet_2, sommet_1)
  nbr_voisins = 0
  for i in range( len(les_voisins_1) ):
    if( les_voisins_1[i] in les_voisins_2 ): 
      nbr_voisins+=1
  return nbr_voisins

#renvoie le degree d'un sommet
def get_degree(matrice, sommet):
  degree = 0
  for i in range(len(matrice[sommet])):
    degree+=matrice[sommet][i]
  return degree

#renvoie le score
def calc_score(matrice, degre, l, m):
  resultat = np.zeros((len(matrice), len(matrice)))
  nbr_conditions = 0
  for i in range(len(matrice)):
    for j in range(len(matrice[i])):
      if( matrice[i][j] == 1 and nbr_voisins_en_commun(matrice, i, j) == l ):
        nbr_conditions+= 1
      elif( matrice[i][j] == 0 and nbr_voisins_en_commun(matrice, i, j) == m ):
        nbr_conditions+= 1
      elif( i == j and matrice[i][j] == 0 and get_degree(matrice, i) == degre):
        nbr_conditions+= 1
      else:
        nbr_conditions+= 0
  return (nbr_conditions/(len(matrice)*len(matrice)))


def ancetre(size, degre, nbr_individu):
  resultat = []
  for i in range (nbr_individu):
    #Etape 1 : on génère alatoirement un graphe k-regulier de n sommets. 
    g = nx.random_regular_graph(degre, size)
    matrice = nx.to_numpy_matrix(g)
    matrice = np.squeeze(np.asarray(matrice))
    resultat.append(matrice)
  return resultat

#renvoie le meilleur score et indivu sur une génération:
def meilleur_score_et_individu(generation, degre, l, m):
  score = 0
  meilleur_individu = []
  for i in range(len(generation)):
    current_score = calc_score(generation[i], degre, l, m)
    if( current_score > score ):
      score = current_score
      meilleur_individu = generation[i]
  return [score, meilleur_individu]


#on tire aléatoirement 2 individus à chaque fois et on les laisse s'affronter: le meilleur est sélectionné.
#à la fin de la fonction, le nombre d'individus est divisé par 2
def selection_par_tournoi(generation, degre, l, m):
  resultat = []
  #on tire aléatoirement 2 individus à chaque fois et on les laisse s'affronter: le meilleur est sélectionné.
  it_liste = random.sample(range(0,len(generation)), len(generation))
  #print(it_liste)
  if( len(it_liste) >= 2 ):
    for i in range(0, len(it_liste)-1, 2):
      score_1 = calc_score(generation[it_liste[i]], degre, l, m)
      score_2 = calc_score(generation[it_liste[i+1]], degre, l, m)
      if( score_1 > score_2 ):
        resultat.append(generation[it_liste[i]])
      else:
        resultat.append(generation[it_liste[i+1]])
    if( len(it_liste)%2 != 0 ):
      resultat.append(generation[it_liste[len(it_liste)-1]])
  elif( len(it_liste) == 1 ):
    resultat.append(generation[0])
  return resultat

#on tire aléatoirement 2 individus à chaque fois et on applique un crosseover
#à la fin de la fonction, le nombre d'individus est le meme.
def croisement_simple(generation, degre, l, m):
  resultat = []
  #on tire aléatoirement 2 individus à chaque fois et on applique un crosseover.
  it_liste = random.sample(range(0,len(generation)), len(generation))
  if( len(it_liste) >= 2 ):
    for i in range(0, len(it_liste)-1, 1):
      #on recupère les 2 parents:
      parent_1 = generation[it_liste[i]]
      parent_2 = generation[it_liste[i+1]]
      enfant_1 = parent_1
      enfant_2 = parent_2
      taille = (len(parent_1)-1)
      #on prends un sommet au hasard, et on fait la mutation:
      sommet_aleatoire = random.randint(0,taille)
      for j in range(sommet_aleatoire+1):
        for k in range(len(enfant_1[taille-j])):
          #enfant 1 :
          #on s'assure de vérifier toutes les propriétés.
          enfant_1[taille-j][k] = parent_2[taille-j][k]
          enfant_1[k][taille-j] = parent_2[taille-j][k]
          #enfant 2 :
          enfant_2[taille-j][k] = parent_1[taille-j][k]
          enfant_2[k][taille-j] = parent_1[taille-j][k]
      resultat.append(enfant_1)
      resultat.append(enfant_2)
    resultat.append(generation[it_liste[len(it_liste)-1]])
  elif( len(it_liste) == 1 ):
    resultat.append(generation[0])
  return resultat


#nombre de génération à créer.
nbr_generation = 1000

#nombre d'individus par génération
nbr_individu = 50

#la première génération
generation = ancetre(SIZE, DEGRE, nbr_individu)
best_score_individu = meilleur_score_et_individu(generation, DEGRE, LAMBDA, MU)


i = 0

bst = best_score_individu
#Tant qu'on a pas trouvé la solution, 
#+on ajoute un nombre maximum de génération à créer.
while( best_score_individu[0] < 1 or i < nbr_generation ):
  generation = ancetre(SIZE, DEGRE, nbr_individu)
  best_score_individu = meilleur_score_et_individu(generation, DEGRE, LAMBDA, MU)

  if( bst[0] < best_score_individu[0] ):
    bst = best_score_individu
  
  #Etape 1 : LA SELECTION dans notre cas, on appliquera la selection par tournoi 
  generation = selection_par_tournoi(generation, DEGRE, LAMBDA, MU)
  
  """
  #Etape 2 : Crossover dans notre cas, on appliquera le croisement simple.
  generation = croisement_simple(generation, DEGRE, LAMBDA, MU)

  #Etape 4 : on recupère le meilleur des individu
  best_score_individu = meilleur_score_et_individu(generation, DEGRE, LAMBDA, MU)

  print(best_score_individu[0])

  #print(best_score_individu)
  """
  i+=1

G = nx.from_numpy_matrix(bst[1]) 
nx.draw_circular(G, with_labels=True)

"""
print(best_score_individu[0]*100)


#print(best_score_individu[1])

"""